<a href="https://colab.research.google.com/github/jeekyung/ESAA/blob/main/%EC%95%99%EC%83%81%EB%B8%94_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___
직접투표: 각 분류기의 예측을 모아 가장 많이 선택된 클래스로 결정(각 예측 결과 클래스 고려)<br/>
간접투표: 각 분류기의 클래스의 확률을 예측하여 이를 평균 내서 확률이 더 높은 클래스로 결정(확률 고려)

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___
학습률을 높여야함. 학습률이 낮아서 예측기가 너무 많아 과대적합된 것이므로 

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [19]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [20]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다. 

In [21]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [6]:
# model fitting
rf_clf = RandomForestClassifier()
xt_clf = ExtraTreesClassifier()
svm_clf = LinearSVC()
mlp_clf = MLPClassifier()

rf_clf.fit(X_train, y_train)
xt_clf.fit(X_train,y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


MLPClassifier()

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [7]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf',rf_clf),('xt',xt_clf),('svm',svm_clf),('mlp',mlp_clf)],
    voting='hard')

In [8]:
# model fitting

voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('xt', ExtraTreesClassifier()),
                             ('svm', LinearSVC()), ('mlp', MLPClassifier())])

In [10]:
#model test
from sklearn.metrics import accuracy_score

y_pred = voting_clf.predict(X_val)
accuracy_score(y_val,y_pred)

0.9689610389610389

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [9]:
# 각 분류 모델 학습 및 성능 확인
from sklearn.metrics import accuracy_score

for clf in (rf_clf, xt_clf, svm_clf, mlp_clf):
  y_pred = clf.predict(X_val)
  print(clf.__class__.__name__, accuracy_score(y_val,y_pred))

RandomForestClassifier 0.965021645021645
ExtraTreesClassifier 0.9688744588744589
LinearSVC 0.8445021645021645
MLPClassifier 0.9596536796536796


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. LinearSVC

In [16]:
# 가장 성능이 낮은 모델 제거

del voting_clf.estimators_[2]

In [17]:
# model fitting
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('xt', ExtraTreesClassifier()),
                             ('svm', LinearSVC()), ('mlp', MLPClassifier())])

In [18]:
# 모델 제거 후 성능 확인
y_pred = voting_clf.predict(X_val)
accuracy_score(y_val,y_pred)


0.9687445887445888

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [11]:
rf_pred = rf_clf.predict(X_val)
xt_pred = xt_clf.predict(X_val)
svm_pred = svm_clf.predict(X_val)
mlp_pred = mlp_clf.predict(X_val)

pred = np.array([rf_pred, xt_pred, svm_pred, mlp_pred])
pred = np.transpose(pred)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [12]:
rf_clf.fit(pred,y_val)

RandomForestClassifier()

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다. 

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [13]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
rf_pred = rf_clf.predict(X_test)
xt_pred = xt_clf.predict(X_test)
svm_pred = svm_clf.predict(X_test)
mlp_pred = mlp_clf.predict(X_test)

pred = np.array([rf_pred, xt_pred, svm_pred, mlp_pred])
pred = np.transpose(pred)

In [ ]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
final = rf_clf.predict(pred)

In [14]:
# model test
accuracy_score(y_test,final)

0.9802164502164502